# **Homework 1 - task 28**
The main goal of this homework is to transform existing evaluation datasets into a format suitable for evaluating the linguistic skills of Large Language Models (LLMs) by reframing tasks as multi-choice Question Answering (QA) tasks, providing effective prompts, and generating distractors where necessary, all formatted in JSON Lines standard for submission.

In [35]:
# from google.colab import drive
# drive.mount('/content/drive')

# **1. Data Loading:**


First we need to download the datasets from evalita to our python eviroment



In [1]:
##### Data Reader -- English/French/Italian
!git clone https://github.com/shammur/SemEval2022Task3.git
%cd ./SemEval2022Task3/
!ls

c:\Users\35193\Desktop\Sapienza\1 year\Second Semester\Multilingual Natural Language Processing\Homework 1\GitHub\NLP_Sapienza\Task_28\Subtask_1\SemEval2022Task3


fatal: destination path 'SemEval2022Task3' already exists and is not an empty directory.
c:\Users\35193\miniconda3\envs\cuda_env\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
'ls' is not recognized as an internal or external command,
operable program or batch file.


# Sub Task 1 

**binary classification sub-task**, which consists in predicting the acceptability label assigned to each sentence of the test set

## 1.1 visualize initial data

In [37]:
import os

# Change the working directory
os.chdir(r'c:\Users\35193\Desktop\Sapienza\1 year\Second Semester\Multilingual Natural Language Processing\Homework 1\GitHub\NLP_Sapienza\Task_28\Subtask_1')

# Verify the current working directory
print("Current working directory:", os.getcwd())

Current working directory: c:\Users\35193\Desktop\Sapienza\1 year\Second Semester\Multilingual Natural Language Processing\Homework 1\GitHub\NLP_Sapienza\Task_28\Subtask_1


In [38]:
def visualize_original_data(file_path):
    try:
        with open(file_path, 'r') as file:
            for i in range(5):  
                line = file.readline()
                print(line.rstrip())
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Check test data
file_path = r'SemEval2022Task3\data\test\official_test_set_with_labels\subtask-1\It-Subtask1-labels.tsv'
visualize_original_data(file_path)


ID	Construction	Sentence	Labels
it_0	prefer	Non amo i campi , preferisco i condomini .	1
it_1	particular	Posso capire le emozioni , e in particolare la tristezza .	1
it_2	generally	Apprezzo i giochi da tavolo , e piÃ¹ in generale il calcio .	0
it_3	generally	Amo i film , e piÃ¹ in generale i western .	0


In [39]:
# Check info in train data
file_path = r'SemEval2022Task3\data\train\train_subtask-1\it\It-Subtask1-fold_0.tsv'
visualize_original_data(file_path)

ID	Sentence	Labels
5491	Amo i cartoni animati , ma non i sussidiari .	1
5084	Apprezzo il vino , ma non il Chianti .	1
1677	Amo i manuali , un tipo interessante di dipinto .	0
1959	Amo gli arbusti , ed anche le querce .	1


In [40]:
# Check info in train data
file_path = r'SemEval2022Task3\data\train\train_subtask-1\it\It-Subtask1-fold_1.tsv'
visualize_original_data(file_path)

ID	Sentence	Labels
2625	Ho incontrato gli astronomi , ed anche gli scienziati .	0
3069	Non amo le carote , preferisco il prosciutto cotto .	1
2414	Si fida dei sensi , ed anche della vista .	0
2239	Amo i granchi , ed anche il pollo .	1


In [41]:
# Check info in train data
file_path = r'SemEval2022Task3\data\train\train_subtask-1\it\It-Subtask1-fold_2.tsv'
visualize_original_data(file_path)

ID	Sentence	Labels
5407	Si fida dell' udito , ma non le dicerie .	1
322	Amo i sussidiari , e piÃ¹ specificamente i thriller .	0
2162	Apprezzo i giochi da tavolo , ed anche il blues .	1
377	Amo le collane , e piÃ¹ specificamente i gioielli .	0


## 1.2 Data Reframing:
  First we need to change the type of the file from xml to json. Then we want to convert that json into the following format:
```JSON
{
    "id":       int,
    "text":     str,
    "choices":  list[str],
    "label":    int
}
```

In [42]:
import csv
import json

def tsv_to_jsonl(input_file, output_file):
    with open(input_file, 'r', newline='') as tsvfile:
        reader = csv.DictReader(tsvfile, delimiter='\t')
        with open(output_file, 'w') as jsonlfile:
            for row in reader:
                json.dump(row, jsonlfile)
                jsonlfile.write('\n')

tsv_file = r'SemEval2022Task3\data\test\official_test_set_with_labels\subtask-1\It-Subtask1-labels.tsv'
jsonl_file = 'It-Subtask1-test.jsonl'
tsv_to_jsonl(tsv_file, jsonl_file)

In [43]:
file1 = r'SemEval2022Task3\data\train\train_subtask-1\it\It-Subtask1-fold_0.tsv'
out_file1 = 'It-Subtask1-fold_0.jsonl'
file2 = r'SemEval2022Task3\data\train\train_subtask-1\it\It-Subtask1-fold_1.tsv'
out_file2 = 'It-Subtask1-fold_1.jsonl'
file3 = r'SemEval2022Task3\data\train\train_subtask-1\it\It-Subtask1-fold_2.tsv'
out_file3 = 'It-Subtask1-fold_2.jsonl'

tsv_to_jsonl(file1, out_file1)
tsv_to_jsonl(file2, out_file2)
tsv_to_jsonl(file3, out_file3)

In [44]:
# join the fold training data to a single file

def join_files(file_paths, output_file):
    with open(output_file, 'w') as outfile:
        for file_path in file_paths:
            with open(file_path, 'r') as infile:
                for line in infile:
                    outfile.write(line)

# Example usage
file1 = 'It-Subtask1-fold_0.jsonl'
file2 = 'It-Subtask1-fold_1.jsonl'
file3 = 'It-Subtask1-fold_2.jsonl'
output_file = 'It-Subtask1-train.jsonl'

join_files([file1, file2, file3], output_file)

## 1.3 Visualize jsonl information

In [45]:
import json

def visualize_data(jsonl_file):
    with open(jsonl_file, 'r') as f:
        line_count = 0
        for line in f:
            data = json.loads(line)
            print(data)
            line_count += 1
            if line_count == 5:
                break

json_file = 'It-Subtask1-test.jsonl'
visualize_data(json_file)


{'ID': 'it_0', 'Construction': 'prefer', 'Sentence': 'Non amo i campi , preferisco i condomini .', 'Labels': '1'}
{'ID': 'it_1', 'Construction': 'particular', 'Sentence': 'Posso capire le emozioni , e in particolare la tristezza .', 'Labels': '1'}
{'ID': 'it_2', 'Construction': 'generally', 'Sentence': 'Apprezzo i giochi da tavolo , e piÃ¹ in generale il calcio .', 'Labels': '0'}
{'ID': 'it_3', 'Construction': 'generally', 'Sentence': 'Amo i film , e piÃ¹ in generale i western .', 'Labels': '0'}
{'ID': 'it_4', 'Construction': 'unlike', 'Sentence': 'A differenza dello shopping , il basket Ã¨ menzionato spesso in questo testo .', 'Labels': '1'}


In [46]:
json_file = 'It-Subtask1-train.jsonl'
visualize_data(json_file)

{'ID': '5491', 'Sentence': 'Amo i cartoni animati , ma non i sussidiari .', 'Labels': '1'}
{'ID': '5084', 'Sentence': 'Apprezzo il vino , ma non il Chianti .', 'Labels': '1'}
{'ID': '1677', 'Sentence': 'Amo i manuali , un tipo interessante di dipinto .', 'Labels': '0'}
{'ID': '1959', 'Sentence': 'Amo gli arbusti , ed anche le querce .', 'Labels': '1'}
{'ID': '731', 'Sentence': 'Uso il poliestere , eccetto il vetro .', 'Labels': '0'}



## 1.4 Format jsonl

In order to get the json file with the desired format:

1. Rename ID to id.
2. Rename Sentence to text.
3. Add a choices key with non acceptable  or aceptable.
4. Rename Labels to label (0- non acceptable and 1 aceptable)

In [47]:
import json

import json

# Rearrange the JSONL file content and write to another JSONL file
def rearrange_jsonl(input_file_path, output_file_path):
    id_counter = 0
    with open(input_file_path, "r") as input_file:
        with open(output_file_path, "w") as output_file:
            for line in input_file:
                data = json.loads(line)
                new_data = {
                    #"id": int(data['id']),
                    "id": id_counter, # set id as an increment int
                    "text": data['Sentence'],
                    "choices": ["non accettabile", "accettabile"],
                    "label": data['Labels']
                }
                id_counter += 1
                json.dump(new_data, output_file)
                output_file.write('\n')

input_jsonl_file = 'It-Subtask1-test.jsonl'
output_jsonl_file = 'PreTENS-task1-test-data.jsonl'
rearrange_jsonl(input_jsonl_file, output_jsonl_file)

input_jsonl_file = 'It-Subtask1-train.jsonl'
output_jsonl_file = 'PreTENS-task1-train-data.jsonl'
rearrange_jsonl(input_jsonl_file, output_jsonl_file)

## 1.5 Visualize final jsonl

In [48]:
json_file = 'PreTENS-task1-test-data.jsonl'
visualize_data(json_file)

{'id': 0, 'text': 'Non amo i campi , preferisco i condomini .', 'choices': ['non accettabile', 'accettabile'], 'label': '1'}
{'id': 1, 'text': 'Posso capire le emozioni , e in particolare la tristezza .', 'choices': ['non accettabile', 'accettabile'], 'label': '1'}
{'id': 2, 'text': 'Apprezzo i giochi da tavolo , e piÃ¹ in generale il calcio .', 'choices': ['non accettabile', 'accettabile'], 'label': '0'}
{'id': 3, 'text': 'Amo i film , e piÃ¹ in generale i western .', 'choices': ['non accettabile', 'accettabile'], 'label': '0'}
{'id': 4, 'text': 'A differenza dello shopping , il basket Ã¨ menzionato spesso in questo testo .', 'choices': ['non accettabile', 'accettabile'], 'label': '1'}


In [49]:
json_file = 'PreTENS-task1-train-data.jsonl'
visualize_data(json_file)


{'id': 0, 'text': 'Amo i cartoni animati , ma non i sussidiari .', 'choices': ['non accettabile', 'accettabile'], 'label': '1'}
{'id': 1, 'text': 'Apprezzo il vino , ma non il Chianti .', 'choices': ['non accettabile', 'accettabile'], 'label': '1'}
{'id': 2, 'text': 'Amo i manuali , un tipo interessante di dipinto .', 'choices': ['non accettabile', 'accettabile'], 'label': '0'}
{'id': 3, 'text': 'Amo gli arbusti , ed anche le querce .', 'choices': ['non accettabile', 'accettabile'], 'label': '1'}
{'id': 4, 'text': 'Uso il poliestere , eccetto il vetro .', 'choices': ['non accettabile', 'accettabile'], 'label': '0'}


## 1.6 Compare distribution of the dataset
Allow to check in information was mantain after the Reframing of the data

In [50]:
import json

def Check_dataset_JSONL(jsonl_file):
    total = 0
    accettabile_count = 0
    non_accettabile_count = 0

    with open(jsonl_file, "r") as file:
        for line in file:
            data = json.loads(line)
            choice = data['choices'][int(data['label'])]  # Convert 'label' to an integer
            if choice == "accettabile":
                accettabile_count += 1
            elif choice == "non accettabile":
                non_accettabile_count += 1
            total += 1

    return accettabile_count, non_accettabile_count, total


In [51]:
def Check_dataset_TSV(tsv_file):
    total = 0
    label_1_count = 0
    label_0_count = 0

    with open(tsv_file, "r") as file:
        next(file)  # Skip header line
        for line in file:
            data = line.strip().split("\t")
            label = int(data[-1])  # Convert label to integer
            if label == 1:
                label_1_count += 1
            elif label == 0:
                label_0_count += 1
            total += 1

    return label_1_count, label_0_count, total

In [52]:
# Check test the dataset distribution 
jsonl_file = "PreTENS-task1-test-data.jsonl"  # Update with your actual file path
accettabile_count, non_accettabile_count, total = Check_dataset_JSONL(jsonl_file)
print("JSONL Output file:")
print("accettabile:", accettabile_count)
print("non accettabile:", non_accettabile_count)
print("Total:", total)

tsv_file = r"SemEval2022Task3\data\test\official_test_set_with_labels\subtask-1\It-Subtask1-labels.tsv"  # Update with your actual file path
label_1_count, label_0_count, total = Check_dataset_TSV(tsv_file)
print("\nTSV Output file:")
print("accettabile:", label_1_count)
print("non accettabile:", label_0_count)
print("Total:", total)


JSONL Output file:
accettabile: 6853
non accettabile: 7707
Total: 14560

TSV Output file:
accettabile: 6853
non accettabile: 7707
Total: 14560


In [53]:
# Check test the dataset distribution 
jsonl_file = "PreTENS-task1-train-data.jsonl" 
accettabile_count, non_accettabile_count, total = Check_dataset_JSONL(jsonl_file)
print("JSONL Output file:")
print("accettabile:", accettabile_count)
print("non accettabile:", non_accettabile_count)
print("Total:", total)

tsv_file_0 = r"SemEval2022Task3\data\train\train_subtask-1\it\It-Subtask1-fold_0.tsv"  
tsv_file_1 = r"SemEval2022Task3\data\train\train_subtask-1\it\It-Subtask1-fold_1.tsv"
tsv_file_2 = r"SemEval2022Task3\data\train\train_subtask-1\it\It-Subtask1-fold_2.tsv"  
label_1_count_0, label_0_count_0, total_0 = Check_dataset_TSV(tsv_file_0)
label_1_count_1, label_0_count_1, total_1 = Check_dataset_TSV(tsv_file_1)
label_1_count_2, label_0_count_2, total_2 = Check_dataset_TSV(tsv_file_2)
# Combine all folds into one single
print("\nTSV Output file:")
print("accettabile:", label_1_count_0 + label_1_count_1 + label_1_count_2)
print("non accettabile:", label_0_count_1 + label_0_count_1 + label_0_count_2)
print("Total:", total_0 + total_1 + total_2)

JSONL Output file:
accettabile: 3029
non accettabile: 2808
Total: 5837

TSV Output file:
accettabile: 3029
non accettabile: 2808
Total: 5837


## **2. Promt formulation**

In order to use this dataset we need to generate three prompts that can be used to get if a text is acceptable within the context of the sentence and then insert them into a json file. The three prompts are:

**Template 1:**


Prompt: "Valuta se il seguente testo è accettabile o meno secondo il contesto: {{text}}"

Translation: "Evaluate whether the following text is acceptable or not according to the context: {{text}}"

**Template 2:**

Prompt: "Determina se la seguente frase è coerente con il contesto: {{text}}"

Translation: "Determine if the following sentence is coherent with the context: {{text}}"

**Template 3:**

Prompt: "Decidi se il testo fornito è congruente con la situazione descritta: {{text}}"

Translation: "Decide if the provided text is congruent with the described situation: {{text}}"


In [54]:
import json

# Function to generate a JSON Lines file with a list of prompts
def generate_prompts(prompts, output_file):
    with open(output_file, 'w') as jsonl_file:
        for prompt in prompts:
            jsonl_file.write(json.dumps({"prompt": prompt}) + '\n')

    with open(output_file, 'r', encoding='utf-8') as jsonl_file:
        print(jsonl_file.read())


prompts = [
    "Valuta se il seguente testo è accettabile o meno secondo il contesto: {{text}}",
    "Determina se la seguente frase è coerente con il contesto: {{text}}",
    "Decidi se il testo fornito è congruente con la situazione descritta: {{text}}"
]

output_file = "PreTENS-task1-prompt.jsonl"
generate_prompts(prompts, output_file)
print(f"JSON Lines file '{output_file}' generated successfully with {len(prompts)} prompts.")



{"prompt": "Valuta se il seguente testo \u00e8 accettabile o meno secondo il contesto: {{text}}"}
{"prompt": "Determina se la seguente frase \u00e8 coerente con il contesto: {{text}}"}
{"prompt": "Decidi se il testo fornito \u00e8 congruente con la situazione descritta: {{text}}"}

JSON Lines file 'PreTENS-task1-prompt.jsonl' generated successfully with 3 prompts.


# **3. Test if all the JSONL files are valid**

In [55]:
import json

def is_jsonl_valid(filename):
    try:
        with open(filename, 'r') as f:
            for line in f:
                json.loads(line)
    except ValueError as e:
        print(f"Invalid JSONL format: {e}")
        return False
    return True

In [56]:
# Train dataset
filename = 'PreTENS-task1-train-data.jsonl' 
if is_jsonl_valid(filename):
    print("The JSONL file is valid.")
else:
    print("The JSONL file is not valid.")

The JSONL file is valid.


In [57]:
# Test dataset
filename = 'PreTENS-task1-test-data.jsonl' 
if is_jsonl_valid(filename):
    print("The JSONL file is valid.")
else:
    print("The JSONL file is not valid.")

The JSONL file is valid.


In [58]:
# Prompt dataset
filename = 'PreTENS-task1-prompt.jsonl'
if is_jsonl_valid(filename):
    print("The JSONL file is valid.")
else:
    print("The JSONL file is not valid.")

The JSONL file is valid.


# **4. Test Prompts with dataset**

In [59]:
# Device agnostic code
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [60]:
# Configuration
runtimeFlag = device #Run on GPU (you can't run GPTQ on cpu)
cache_dir = None # by default, don't set a cache directory. This is automatically updated if you connect Google Drive.
scaling_factor = 1.0 # allows for a max sequence length of 16384*6 = 98304! Unfortunately, requires Colab Pro and a V100 or A100 to have sufficient RAM.

## 3.1 Install

In [61]:
!pip install -q -U transformers peft accelerate optimum

!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/


In [62]:
!pip install -q -U pdfminer.six # could maybe add pre-built wheels to speed this up.

ERROR: Invalid requirement: '#'


In [63]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

## 3.2 Load Model


In [64]:
import transformers
import torch
import json
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer_llama = AutoTokenizer.from_pretrained(model_id)
model_llama = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)

# **4. Evaluate Homework prompts**

## 4.1. Test response of Promtps

In [65]:
import json
import torch

def visualize_prompt_response(jsonl_file):
    # Load your JSON file and process each pair
    with open(jsonl_file, 'r') as file:
        count = 0  # Counter to track processed pairs
        for line in file:
            if count >= 5:  # Break the loop if 5 pairs have been processed
                break
            pair = json.loads(line)
            #id = pair['id']
            text = pair['text']
            choices = pair['choices']
            label_index = int(pair['label'])  # Convert label_index to an integer
            label = choices[label_index]  # Get the actual label using label_index

            # Format the prompt with text and hypothesis
            prompt_formatted = f"Valuta se il seguente testo è accettabile o meno secondo il contesto: {text}"

            # Assuming you have tokenizer_llama and model_llama defined elsewhere
            input = tokenizer_llama(prompt_formatted, return_tensors="pt").to(device)
            output = model_llama(**input)
            logits = output.logits[0].detach().cpu().numpy()

            # Obtain the predicted label directly from logits
            probabilities = torch.softmax(torch.tensor(logits), -1).detach().cpu().numpy()
            print(f"Probabilities: {probabilities}")
            prediction_index = 1 if probabilities[1] > probabilities[0] else 0
            prediction_label = choices[prediction_index]
            prediction_confidence = probabilities[prediction_index] * 100
            prediction = {"label": prediction_label, "confidence": round(prediction_confidence, 1)}

            # Compare prediction with actual label
            is_correct = prediction_label == label

            # Visualize the formatted prompt and the prediction details
            print("Prompt:", prompt_formatted)
            print("Prediction:", prediction)
            print("Actual Label:", label)
            print("Correct Prediction:", is_correct)
            print()
            count += 1  # Increment the counter for processed pairs

jsonl_file = 'PreTENS-task1-train-data.jsonl'
visualize_prompt_response(jsonl_file)


Probabilities: [9.091138e-04 9.982749e-01 8.160408e-04]
Prompt: Valuta se il seguente testo è accettabile o meno secondo il contesto: Amo i cartoni animati , ma non i sussidiari .
Prediction: {'label': 'accettabile', 'confidence': 99.8}
Actual Label: accettabile
Correct Prediction: True

Probabilities: [2.1190660e-03 9.9753678e-01 3.4415934e-04]
Prompt: Valuta se il seguente testo è accettabile o meno secondo il contesto: Apprezzo il vino , ma non il Chianti .
Prediction: {'label': 'accettabile', 'confidence': 99.8}
Actual Label: accettabile
Correct Prediction: True

Probabilities: [0.00779086 0.9831966  0.00901253]
Prompt: Valuta se il seguente testo è accettabile o meno secondo il contesto: Amo i manuali , un tipo interessante di dipinto .
Prediction: {'label': 'accettabile', 'confidence': 98.3}
Actual Label: non accettabile
Correct Prediction: False

Probabilities: [0.0090381 0.986324  0.0046379]
Prompt: Valuta se il seguente testo è accettabile o meno secondo il contesto: Amo gli a

## 4.2. Compare the diferent prompts

In [66]:
# Prevent CUDA from running out of memory 
torch.cuda.empty_cache()

In [67]:
import json

total_correct = 0
total_pairs = 0

# Load the JSONL file and process each pair
with open("PreTENS-task1-train-data.jsonl", 'r') as file:
        for line in file:
            pair = json.loads(line)
            id = pair['id']
            text = pair['text']
            choices = pair['choices']
            label_index = int(pair['label'])  # Convert label_index to an integer
            label = choices[label_index]  # Get the actual label using label_index
    
            # Format the prompt with actual text and hypothesis
            prompt_formatted = f"Valuta se il seguente testo \u00e8 accettabile o meno secondo il contesto: {text}"

            input = tokenizer_llama(prompt_formatted, return_tensors="pt").to(device)
            output = model_llama(**input)
            logits = output.logits[0].detach().cpu().numpy()

            # Obtain the predicted label directly from logits
            probabilities = torch.softmax(torch.tensor(logits), -1).detach().cpu().numpy()
            prediction_index = 1 if probabilities[1] > probabilities[0] else 0
            prediction_label = choices[prediction_index]

            # Compare prediction with actual label
            is_correct = prediction_label == label

            # Accumulate statistics
            total_correct += is_correct
            total_pairs += 1

# Calculate overall statistics only if at least one pair was processed
if total_pairs > 0:
    accuracy = total_correct / total_pairs * 100

    # Print statistics
    print("Overall Statistics:")
    print("Total Pairs:", total_pairs)
    print("Total Correct:", total_correct)
    print("Accuracy:", round(accuracy, 2), "%")
else:
    print("No pairs found in the JSONL file.")


Overall Statistics:
Total Pairs: 5837
Total Correct: 3029
Accuracy: 51.89 %


In [71]:
import json
import torch

def prompt_testing(prompt, file_path):
    total_correct = 0
    total_pairs = 0
    total_confidence = 0

    # Load the JSONL file and process each pair
    with open(file_path, 'r') as file:
        for line in file:
            pair = json.loads(line)
            #id = pair['id']
            text = pair['text']
            choices = pair['choices']
            label_index = int(pair['label'])  
            label = choices[label_index]  
            
            # Format with text and hypothesis 
            prompt_formatted = prompt.replace('{{text}}', text)
            input = tokenizer_llama(prompt_formatted, return_tensors="pt").to(device)
            output = model_llama(**input)
            logits = output.logits[0].detach().cpu().numpy()

            # Obtain the predicted label directly from logits
            probabilities = torch.softmax(torch.tensor(logits), -1).detach().cpu().numpy()
            prediction_index = 1 if probabilities[1] > probabilities[0] else 0
            prediction_label = choices[prediction_index]

            # Compare prediction with actual label
            is_correct = prediction_label == label

            # Accumulate statistics
            total_correct += is_correct
            total_pairs += 1
            total_confidence += probabilities[prediction_index]

    # Calculate statistics 
    if total_pairs > 0:
        accuracy = total_correct / total_pairs * 100
        mean_confidence = total_confidence / total_pairs

        # Print statistics
        print("Total Pairs:", total_pairs)
        print("Total Correct:", total_correct)
        print("Accuracy:", round(accuracy, 2), "%")
        print("Mean Confidence:", round(mean_confidence, 2))
    else:
        print("No pairs found in the JSONL file.")

# Load prompts from the JSONL file and test each prompt
with open('PreTENS-task1-prompt.jsonl', 'r') as prompts_file:
    prompt_data = []
    for line in prompts_file:
        prompt_item = json.loads(line)
        prompt_data.append(prompt_item)

    for prompt_item in prompt_data:
        prompt = prompt_item["prompt"]
        print("\nPrompt:", prompt)
        file_path = 'PreTENS-task1-train-data.jsonl'
        prompt_testing(prompt, file_path)




Prompt: Valuta se il seguente testo è accettabile o meno secondo il contesto: {{text}}
Total Pairs: 5837
Total Correct: 3029
Accuracy: 51.89 %
Mean Confidence: 0.92

Prompt: Determina se la seguente frase è coerente con il contesto: {{text}}
Total Pairs: 5837
Total Correct: 3029
Accuracy: 51.89 %
Mean Confidence: 0.97

Prompt: Decidi se il testo fornito è congruente con la situazione descritta: {{text}}
Total Pairs: 5837
Total Correct: 3029
Accuracy: 51.89 %
Mean Confidence: 0.8


## 4.3 Check the difference of accuracy between test and trainind dataset

In [72]:
with open('PreTENS-task1-prompt.jsonl', 'r') as prompts_file:
    prompt_data = []
    for line in prompts_file:
        prompt_item = json.loads(line)
        prompt_data.append(prompt_item)

    for prompt_item in prompt_data:
        prompt = prompt_item["prompt"]
        print("\nPrompt:", prompt)
        file_path = 'PreTENS-task1-test-data.jsonl'
        prompt_testing(prompt, file_path)


Prompt: Valuta se il seguente testo è accettabile o meno secondo il contesto: {{text}}
Total Pairs: 14560
Total Correct: 6851
Accuracy: 47.05 %
Mean Confidence: 0.92

Prompt: Determina se la seguente frase è coerente con il contesto: {{text}}
Total Pairs: 14560
Total Correct: 6852
Accuracy: 47.06 %
Mean Confidence: 0.97

Prompt: Decidi se il testo fornito è congruente con la situazione descritta: {{text}}
Total Pairs: 14560
Total Correct: 6848
Accuracy: 47.03 %
Mean Confidence: 0.79
